# Homework 1 - Group A4

## FINM 25000 - 2025

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

## HBS Case

### *The Harvard Management Company and Inflation-Indexed Bonds*

### Notation
(Hidden LaTeX commands)

$$\newcommand{\mux}{\tilde{\boldsymbol{\mu}}}$$
$$\newcommand{\wtan}{\boldsymbol{\text{w}}^{\text{tan}}}$$
$$\newcommand{\wtarg}{\boldsymbol{\text{w}}^{\text{port}}}$$
$$\newcommand{\mutarg}{\tilde{\boldsymbol{\mu}}^{\text{port}}}$$
$$\newcommand{\wEW}{\boldsymbol{\text{w}}^{\text{EW}}}$$
$$\newcommand{\wRP}{\boldsymbol{\text{w}}^{\text{RP}}}$$
$$\newcommand{\wREG}{\boldsymbol{\text{w}}^{\text{REG}}}$$

In [4]:
import pandas as pd
import numpy as np

***

# 1. HMC's Approach

**Section 1 is not graded**, and you do not need to submit your answers. But you are encouraged to think about them, and we will discuss them.

### 1. 
There are thousands of individual risky assets in which HMC can invest.  Explain why MV optimization across 1,000 securities is infeasible.

Answer:
Determinant of cov matrix need to not be zero to find the inverse, thus, the inverse of the covariance matrix can and will be computed.

Statistically speaking, the inversion of the covariance is not feasible. The inversion of the cov matrix (according to mathematician is invertible as long as the determinants are not zero). Statistically there will be a lot of noise in the covariance matrix, and thus the inversion will not be feasible. A tiny determinant will explode any level of uncertainty in the inputs.

There is noise in the data, the optimization will take any noise seriously (financial data is incredibly noisey.)

### 2.
Rather than optimize across all securities directly, HMC runs a two-stage optimization.
1. They build asset class portfolios with each one optimized over the securities of the specific asset class.  
2. HMC combines the asset-class portfolios into one total optimized portfolio.

In order for the two-stage optimization to be a good approximation of the full MV-optimization on all assets, what must be true of the partition of securities into asset classes?

Answer:
When making the sub portfolios, you would like the sub portfolios to have no correlation with each other. 


### 3.
Should TIPS form a new asset class or be grouped into one of the other 11 classes?

Answer:
You need to look at the cross correlations with the other asset classes.

### 4. 
Why does HMC focus on real returns when analyzing its portfolio allocation? Is this just a matter of scaling, or does using real returns versus nominal returns potentially change the MV solution?

Answer:
Does any large asset manager want to make sure they are getting high nominal or high real returns? You may think inflation will dig into either.

It is rather case by case. If inflation is high, TIPS will perform well. If inflation is lower, regular bonds will outperform. 

Question is your objective. If it is nominal or post-inflation adjusted return.

Subtracting inflation does impact covariance matrix.

Currency should also be written in the covariance matrix you are trying to optimize.


SKIPPED THE FOLLOWING


### 5.
The case discusses the fact that Harvard places bounds on the portfolio allocation rather than implementing whatever numbers come out of the MV optimization problem.

How might we adjust the stated optimization problem in the lecture notes to reflect the extra constraints Harvard is using in their bounded solutions given in Exhibits 5 and 6?


### 6. 
Exhibits 5 shows zero allocation to domestic equities and domestic bonds across the entire computed range of targeted returns, (5.75% to 7.25%). Conceptually, why is the constraint binding in all these cases? What would the unconstrained portfolio want to do with those allocations and why?

### 7.
Exhibit 6 changes the constraints, (tightening them in most cases.) How much deterioration do we see in the mean-variance tradeoff that Harvard achieved?

***

# 2 Mean-Variance Optimization

<i>This section is graded for a good-faith effort by your group. Submit your write-up- along with your supporting code. </i>

### Data
You will need the file in the github repo, `data/multi_asset_etf_data.xlsx`.
- The time-series data gives monthly returns for the 11 asset classes and a short-term Treasury-bill fund return, ("SHV",) which we consider as the risk-free rate.
- The data is provided in total returns, (in which case you should ignore the SHV column,) as well as excess returns, (where SHV has been subtracted from the other columns.)
- These are nominal returns-they are not adjusted for inflation, and in our calculations we are not making any adjustment for inflation.
- The exhibit data that comes via Harvard with the case is unnecessary for our analysis.

### Model
We are going to analyze the problem in terms of **excess** returns.
- Thus, you will focus on the `Excess Returns` section of the lecture notes, especially the formulas on slide 50.
- Be sure to use the`excess returns` tab of the data.

### Format
In the questions below, **annualize the statistics** you report.
- Annualize the mean of monthly returns with a scaling of 12.
- Annualize the volatility of monthly returns with a scaling of $\sqrt{12}$
- The Sharpe Ratio is the mean return divided by the volatility of returns. Accordingly, we can annualize the Sharpe Ratio with a scaling of $\sqrt{12}$
- Note that we are not scaling the raw timeseries data, just the statistics computed from it (mean, vol, Sharpe). 

### Footnotes

#### Data File
* The case does not give time-series data, so this data has been compiled outside of the case, and it intends to represent the main asset classes under consideration via various ETFs. For details on the specific securities/indexes, check the “Info” tab of the data.

#### Risk-free rate
* In the lecture-note we considered a constant risk-free rate. It is okay that our risk-free rate changes over time, but the assumption is that investors know it’s value one-period ahead of time. Thus, at any given point in time, it is a risk-free rate for the next period. (This is often discussed as the "bank account" or "money market account" in other settings.

### 1. Summary Statistics
* Calculate and display the mean and volatility of each asset’s excess return. (Recall we use volatility to refer to standard deviation.)
* Which assets have the best and worst Sharpe ratios? Recall that the Sharpe Ratio is simply the ratio of the mean-to-volatility of excess returns:
$$\text{sharpe ratio of investment }i = \frac{\mux_i}{\sigma_i}$$

Be sure to annualize all three statss (mean, vol, Sharpe).
* mean is scaled by `12`
* vol is scaled by `sqrt(12)`
* Sharpe is scaled by `sqrt(12)`

In [5]:
#HW dataframe
excess_returns_df = pd.read_excel(io = './multi_asset_etf_data.xlsx', 
                                  sheet_name='excess_returns',
                                  index_col=0,
                                  parse_dates=[0])

excess_returns_df.head(5)
# tickers = ['BWX', 'DBC', 'EEM', 'EFA', 'HYG', 'IEF', 'IYR', 'PSP', 'QAI', 'SPY', 'TIP']

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2011-02-28,0.007027,0.042120,-0.000027,0.035908,0.014763,-0.001674,0.045614,0.040556,0.002993,0.035147,0.007618
2011-03-31,0.008755,0.026909,0.063224,-0.023555,0.000752,-0.001218,-0.010607,0.016995,0.005849,0.000448,0.012231
2011-04-30,0.048760,0.045514,0.027283,0.056214,0.015932,0.018369,0.046589,0.058627,0.018989,0.028917,0.023735
2011-05-31,-0.012945,-0.051124,-0.028800,-0.021461,0.001933,0.025654,0.010733,-0.040965,0.000600,-0.010615,0.003259
2011-06-30,0.000230,-0.042318,-0.009027,-0.011781,-0.005378,-0.004716,-0.030733,-0.042254,-0.010449,-0.016542,0.007876


In [6]:
def performance_summary(rets, adj_factor=12):
    summary = {}
    summary['Annualized Mean'] = rets.mean() * adj_factor
    summary['Annualized Volatility'] = rets.std() * np.sqrt(adj_factor)
    summary['Annualized Sharpe Ratio'] = (
        summary['Annualized Mean'] / summary['Annualized Volatility']
        )
    return pd.DataFrame(summary, index=rets.columns)

metrics = performance_summary(excess_returns_df).sort_values(
    'Annualized Sharpe Ratio', ascending=False
)


display(metrics)

,Annualized Mean,Annualized Volatility,Annualized Sharpe Ratio
SPY,0.128141,0.142839,0.897103
HYG,0.041371,0.075928,0.544873
IYR,0.074916,0.168675,0.444143
PSP,0.092561,0.213370,0.433804
EFA,0.061775,0.150903,0.409372
TIP,0.020502,0.051115,0.401091
QAI,0.019327,0.049073,0.393838
IEF,0.016404,0.063442,0.258569
EEM,0.029339,0.176164,0.166542
DBC,-0.005292,0.166553,-0.031774


In [7]:
print(f'Max sharpe ratio: {metrics['Annualized Sharpe Ratio'].idxmax()} at {metrics['Annualized '
                                                                                    'Sharpe Ratio']
                                                                                    .max()}')
print()
print(f'Min sharpe ratio: {metrics['Annualized Sharpe Ratio'].idxmin()} at {metrics['Annualized '
                                                                                    'Sharpe Ratio']
                                                                                    .min()}')

SyntaxError: unterminated string literal (detected at line 1) (3749496906.py, line 1)

Based on the above data, SPY (SPDR's S&P 500 index) has the best Sharpe ratio and BWX (SPDR's Bloomberg International Treasury Bond ETF) has the worst Sharpe ratio. In other words, SPY has the greatest return per unit of risk ratio and International Bonds have the lowest return per unit of risk.

### 2. Descriptive Analysis
* Calculate the correlation matrix of the returns. Which pair has the highest correlation? And the lowest?
* How well have TIPS done in our sample? Have they outperformed domestic bonds? Foreign bonds?

In [ ]:
corr_matrix = excess_returns_df.corr()
corr_matrix

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
BWX,1.000000,0.191116,0.621673,0.602820,0.602555,0.580891,0.552557,0.526692,0.630276,0.439994,0.675151
DBC,0.191116,1.000000,0.511667,0.500922,0.461887,-0.300207,0.280518,0.453303,0.475311,0.432162,0.109006
EEM,0.621673,0.511667,1.000000,0.819925,0.691167,0.026704,0.584063,0.750109,0.774697,0.687751,0.378792
EFA,0.602820,0.500922,0.819925,1.000000,0.787191,0.042639,0.699292,0.895320,0.847864,0.845863,0.394821
HYG,0.602555,0.461887,0.691167,0.787191,1.000000,0.187258,0.739356,0.812157,0.807893,0.793518,0.538648
IEF,0.580891,-0.300207,0.026704,0.042639,0.187258,1.000000,0.316532,0.022436,0.179761,0.000815,0.754102
IYR,0.552557,0.280518,0.584063,0.699292,0.739356,0.316532,1.000000,0.749836,0.718529,0.754711,0.598742
PSP,0.526692,0.453303,0.750109,0.895320,0.812157,0.022436,0.749836,1.000000,0.873395,0.891687,0.408005
QAI,0.630276,0.475311,0.774697,0.847864,0.807893,0.179761,0.718529,0.873395,1.000000,0.866845,0.516670
SPY,0.439994,0.432162,0.687751,0.845863,0.793518,0.000815,0.754711,0.891687,0.866845,1.000000,0.381625


PSP and EFA have the highest correlation at 0.895320

SPY and IEF have the lowest correlation by absolute value at 0.000815

IEF and DBC have the lower correlation by magnitude at -0.300207

In [ ]:
print('Ranking of annualized mean returns (ascending):')
print()
print(metrics['Annualized Mean'].sort_values())

Ranking of annualized mean returns (ascending):

BWX   -0.007716
DBC   -0.005292
IEF    0.016404
QAI    0.019327
TIP    0.020502
EEM    0.029339
HYG    0.041371
EFA    0.061775
IYR    0.074916
PSP    0.092561
SPY    0.128141
Name: Annualized Mean, dtype: float64


In [ ]:
print('Ranking of volatility of returns (ascending):')
print()
print(metrics['Annualized Volatility'].sort_values())

Ranking of volatility of returns (ascending):

QAI    0.049073
TIP    0.051115
IEF    0.063442
HYG    0.075928
BWX    0.082789
SPY    0.142839
EFA    0.150903
DBC    0.166553
IYR    0.168675
EEM    0.176164
PSP    0.213370
Name: Annualized Volatility, dtype: float64


In [ ]:
print('Sharpe Ratio for each asset (annualized):')
print()
print(metrics['Annualized Sharpe Ratio'].sort_values())

Sharpe Ratio for each asset (annualized):

BWX   -0.093202
DBC   -0.031774
EEM    0.166542
IEF    0.258569
QAI    0.393838
TIP    0.401091
EFA    0.409372
PSP    0.433804
IYR    0.444143
HYG    0.544873
SPY    0.897103
Name: Annualized Sharpe Ratio, dtype: float64


TIPS rank 7th out of the 11 assets in terms of mean returns. The four assets with lower mean returns include QAI, IEF, DBC, and BWX. In other words, TIPS (Treasury Inflation Protected Securities) yield a higher return than an ETF that aims to track the performance of hedgefunds (QAI), the 7-10 year US treasury bonds (IEF), a portfolio of exchange traded futures of an assortment of different commodities (DBC), and an ETF that tracks the fixed-rate local currency sovereign debt of investment grade countries outside the US (BWX). Alternatively, the TIPS that provide a practically riskless inflation adjusted return yield a higher return than a portfolio of hedgefunds, 7-10 year US Treasuries, commodities, and investment grade sovereign debt outside the US. 


TIPS rank 10th out of the 11 assets in terms of volatility of returns. The only asset with lower volatility is QAI, the ETF that tracks an assortment of hedgefunds' performance. This ranking makes sense due to the fact that TIPS provide a practically risk free return and therefore should be on the lower end of volatilities.

Lastly, TIPS rank 6th out of the 11 assets in terms of sharpe ratio. The five assets with a lower sharpe ratio are QAI, IEF, EEM, DBC, and BWX. In other words, TIPS provide a higher return per unit of risk compared to hedge funds, the 7-10 year Treasury Bonds, a performance tracker of the emerging markets, commodities, and foreign debt.

Based on the above data, TIPS have a return of 2.0502% while IEF and BWX have returns of 1.6404% and -0.7716% respectively. Additionally, TIPS have a vol of 5.1115% while IEF and BWX have vols of 6.3442% and 8.2789% respectively. Lastly, TIPS provide a sharpe ratio of 0.401091 while IEF and BWX have sharpe ratios of 0.258569 and 0.093202 respectively. Therefore, in terms of pure returns, volatility, and sharpe ratios, TIPS outperform domestic and foreign debt (IEF and BWX respectively.)

### 3. The MV frontier.
* Compute and display the weights of the tangency portfolios: $\wtan$.
* Does the ranking of weights align with the ranking of Sharpe ratios?
* Compute the mean, volatility, and Sharpe ratio for the tangency portfolio corresponding to
$\wtan$.

In [ ]:
# Main Strategy:

def tan_portfolio_weights(dataframe, title):
    """
    This function calculates the tengency portfolio weights.

    Args:
        dataframe: a dataframe of excess returns

    Returns:
        A vector of tangency portfolio weights.
    """
    mean_returns = dataframe.mean()
    Sigma = dataframe.cov()

    inv_Sigma = np.linalg.inv(Sigma) # inverse of sigma

    ones = np.ones(mean_returns.shape)

    unscaled_portfolio_weights = inv_Sigma @ mean_returns # cov matrix times mean returns vector
    sum_unscaled_portfolio_weights = ones.T @ inv_Sigma @ mean_returns # scalar
    weights = unscaled_portfolio_weights / sum_unscaled_portfolio_weights #normalization of weights

    #convert to dataframe
    w_tan_df = pd.DataFrame(weights, 
                        index = dataframe.columns, 
                        columns = [title]
                        )
    return w_tan_df

w_tan_df = tan_portfolio_weights(excess_returns_df, 'Tangency Portfolio')



print("Tangency Portfolio Weights: ")
display(w_tan_df.sort_values(by='Tangency Portfolio', ascending=False))
print()
print("Annualized Sharpe Ratio of each asset (annualized):")
display(metrics['Annualized Sharpe Ratio'].sort_values(ascending=False))

w_tan_returns = excess_returns_df @ w_tan_df

tan_summary = performance_summary(w_tan_returns)
print()
display(tan_summary)

Tangency Portfolio Weights: 


,Tangency Portfolio
SPY,10.729782
IEF,9.010591
HYG,2.634922
TIP,1.330595
EEM,0.853591
EFA,0.385285
DBC,-0.112341
PSP,-1.716842
IYR,-2.382146
BWX,-6.117430



Annualized Sharpe Ratio of each asset (annualized):


SPY    0.897103
HYG    0.544873
IYR    0.444143
PSP    0.433804
EFA    0.409372
TIP    0.401091
QAI    0.393838
IEF    0.258569
EEM    0.166542
DBC   -0.031774
BWX   -0.093202
Name: Annualized Sharpe Ratio, dtype: float64

,Annualized Mean,Annualized Volatility,Annualized Sharpe Ratio
Tangency Portfolio,1.155145,0.744433,1.55171


In [ ]:
print(f'Determinant: {np.linalg.det(excess_returns_df.cov())}')

Determinant: 2.981422358888704e-39


Note: the determinant of the covariance is extremely tiny. As a result, a large number scales the weights to be outrageously large. 1000% long and -1300% short in assets is simply unrealistic.

Note: The tagency portfolio also seems to favor some assets way higher others. The weights do not line up with the sharpe ratios. The optimizer doesn't care about stand alone vol. It cares about the covariances, which is a rescaling of correlations. SPY and QAI are inverses because the two are highly correlated and a long SPY and short QAI would yield a high return and a lower vol. 

Note: Main idea from lecture 1: individual performance is not indicative of portfolio performance.

In [ ]:
# Alternative strategy we used:

merged = pd.merge(metrics, w_tan_df, left_index=True, right_index=True)

def tan_metrics(df_returns,df):
  """
  df_ereturns: original data
  df: tangency portfolio weights merged with metrics

  Given a dataframe of metrics including tangency weighted portfolios, computes
  the mean, volatility, and Sharpe ratio wrt the tangency weight.
  """
  w = df[list(df.columns)[3]].values
  mu = df["Annualized Mean"].values
  Sigma = df_returns.cov().values

  assert w.shape[0] == Sigma.shape[0], "Mismatch: weights and covariance matrix size"

  mean_portfolio = w @ mu
  vol_portfolio = np.sqrt(w.T @ Sigma @ w)
  sharpe_portfolio = mean_portfolio / vol_portfolio

  return pd.DataFrame({
      "Annualized Mean": [mean_portfolio],
      "Annualized Volatility": [vol_portfolio],
      "Annualized Sharpe Ratio": [sharpe_portfolio]
    })

tan_metrics(excess_returns_df, merged).head()

,Annualized Mean,Annualized Volatility,Annualized Sharpe Ratio
0,1.155145,0.466146,2.478073


No, the ranking of the weights of the tangency portfolio do not align with the ranking of the sharpe ratios of each asset. While SPY ranks #1 in both instances, from there on out, the ordering is different.

### 4. TIPS
Assess how much the tangency portfolio (and performance) change if...
* TIPS are dropped completely from the investment set.
* The expected excess return to TIPS is adjusted to be 0.0012 higher than what the historic sample shows.

Based on the analysis, do TIPS seem to expand the investment opportunity set, implying that Harvard should consider them as a separate asset?

Q1

In [ ]:
# Dropped TIPS main strategy:

new_tickers = ['BWX', 'DBC', 'EEM', 'EFA', 'HYG', 'IEF', 'IYR', 'PSP', 'QAI', 'SPY']
w_tan_no_tips_df = tan_portfolio_weights(excess_returns_df[new_tickers], 
                                         'Tangency Portfolio (with no TIPS)')

print("Tangency Portfolio Weights: ")
display(w_tan_no_tips_df.sort_values(by='Tangency Portfolio (with no TIPS)', ascending=False))

w_tan_returns_no_tips = excess_returns_df[new_tickers] @ w_tan_no_tips_df

tan_summary_no_tips = performance_summary(w_tan_returns_no_tips)
print()
display(tan_summary_no_tips)

Tangency Portfolio Weights: 


,Tangency Portfolio (with no TIPS)
SPY,14.661934
IEF,13.355289
HYG,3.744656
EEM,1.215107
EFA,0.415306
DBC,-0.048440
PSP,-2.284411
IYR,-3.167946
BWX,-8.320309
QAI,-18.571185


,Annualized Mean,Annualized Volatility,Annualized Sharpe Ratio
Tangency Portfolio (with no TIPS),1.570867,1.013288,1.550267


In [ ]:
# Dropped TIPS alternative strategy:

dropped_tips_df = excess_returns_df.drop(columns=['TIP'])

metrics_no_tips = performance_summary(dropped_tips_df).sort_values('Annualized Sharpe Ratio', 
                                                                   ascending=False)

wtan_no_tip = tan_portfolio_weights(dropped_tips_df, 
                                    'Tangency Portfolio (with no TIPS)')["Tangency Portfolio (with "
                                    "no TIPS)"]

wtan_no_tip = pd.merge(metrics_no_tips, 
                       wtan_no_tip, 
                       left_index=True, 
                       right_index=True)

tan_metrics(dropped_tips_df, wtan_no_tip).head()

,Annualized Mean,Annualized Volatility,Annualized Sharpe Ratio
0,1.570867,0.538566,2.916757


As we can observe from Part 3, the annualized sharpe ratio of the original tangency portfolio was 1.55171 and the annualized sharpe ratio of the tangency portfolio without TIPS was 1.550267 (slightly lower). Therefore, with TIPS in the portfolio, the IOS is marginally different. However, it is also important to note that the tangency portfolio without TIPS yields a higher annualized mean return (157.0867% instead of 115.145%) and a lower vol (46.6146% versus 100.328%).

Q2

In [ ]:
# Adjusted TIPS main strategy

adj_excess_returns_df = excess_returns_df.copy()
adj_excess_returns_df['TIP'] = adj_excess_returns_df['TIP'] + 0.0012

w_tan_adj_df = tan_portfolio_weights(adj_excess_returns_df, 
                                         'Tangency Portfolio (with adjusted TIPS)')

print("Tangency Portfolio Weights: ")
display(w_tan_adj_df.sort_values(by='Tangency Portfolio (with adjusted TIPS)', ascending=False))

w_tan_returns_adj = excess_returns_df @ w_tan_adj_df

tan_summary_adj_tips = performance_summary(w_tan_returns_adj)

print()
display(tan_summary_adj_tips)

Tangency Portfolio Weights: 


,Tangency Portfolio (with adjusted TIPS)
TIP,3.914098
SPY,3.095060
IEF,0.574864
HYG,0.480247
EFA,0.326997
EEM,0.151667
DBC,-0.236412
PSP,-0.614841
IYR,-0.856425
BWX,-1.840290


,Annualized Mean,Annualized Volatility,Annualized Sharpe Ratio
Tangency Portfolio (with adjusted TIPS),0.347972,0.239751,1.45139


Note: the above shows that if we have the inptus off by 0.0012 a year, the advice would be unstable. The math is beautiful but some uncertainty on the inputs leads to drastic uncertainty on the outputs. WORRISOME! but we will fix this in second have of lecture 2

In [ ]:
# Adjusted TIPS alternative strategy

adj_data = excess_returns_df.copy()
adj_data['TIP'] = adj_data['TIP'] + 0.0012

metrics_adj = performance_summary(adj_data).sort_values('Annualized Sharpe Ratio', 
                                                                   ascending=False)

w_tan_adj = tan_portfolio_weights(adj_data, 
                                    'Tangency Portfolio (with adjusted TIPS)')["Tangency Portfolio "
                                                                               "(with adjusted TIPS)"]

w_tan_adj = pd.merge(metrics_adj, 
                       w_tan_adj, 
                       left_index=True, 
                       right_index=True)

tan_metrics(adj_data, w_tan_adj).head()

,Annualized Mean,Annualized Volatility,Annualized Sharpe Ratio
0,0.404335,0.225329,1.794422


As we can observe from Part 3, the annualized sharpe ratio of the original tangency portfolio was 1.55171 and the annualized sharpe ratio of the tangency portfolio with the adjusted TIPS is 1.45139 (slightly lower). Therefore, with adjusted TIPS in the portfolio, the IOS is slightly smaller. It is also important to note that the tangency portfolio with the adjusted TIPS yields a dramatically lower annualized mean return (40.4335%  instead of 115.145%) and a lower vol (22.5329% instead of 100.328%).

Therefore, based on the above data analysis it is not clear whether or not TIPS truly expands or tighten the IOS. The presence of TIPS in the original portfolio does lead to a slightly higher Sharpe and TIPS removal did lead to a significantly lower vol and higher return. This could mean that they may be the source of the volatility in the portfolio. Additionally, when their expected return was artificially increased, the portfolio's performance worsened. Thus, it is not clear that TIPS expands the IOS in a beneficial way for Harvard. More analysis on correlation would provide more detailed information on whether or not to include the TIPS. We can assume that TIPS likely have a higher variance inflation factor with IEF (The 7-10 Year Treasury bond ETF) since TIPS are simply the inflation adjusted versions of such bonds. In fact, the correlation between IEF and TIP is 0.754102 which is quite high. The fact that TIPS are highly correlated with will conflate the covariances of the returns of the portfolio and reduce diversification benefits which would support the exclusion of TIPS from Harvard's portfolio as a separate asset.

***

# 3. Allocations

<i>This section is graded for a good-faith effort by your group. Submit your write-up- along with your supporting code.

* Continue with the same data file as the previous section.

* Suppose the investor has a targeted mean excess return (per month) of $\mutarg$ = 0.01.

In [ ]:
def mv_portfolio(mean_returns, cov_matrix, target=None):
    """
    This function calculates the weight of the mean-variance portfolio.

    If the target mean return is not specified, then the function will return the tangency portfolio. 
    If the target is specified, then the function will return the MV-efficient portfolio wiht the 
    target return.

    Key assumption: there is access to a risk-free asset and we are using excess returns

    Args: 
        mean_returns: A vector of mean returns
        cov_matrix: Covariance matrix of returns
        target (optional): The target mean return. Defaults to none. Note: this must be adjusted for 
                           the annualization the same time-frequency as the mean returns. If the mean
                           returns are monthly, the target must be monthly as well.

    Returns:
        A vector of MV weights.
    """
    w_tan = tan_portfolio_weights(excess_returns_df, 'Tangency Portfolio')

    if target is None:
        return w_tan
    
    else:
        ones = np.ones(mean_returns.shape) # a vector of ones the size of the number of assets
        cov_inv = np.linalg.inv(cov_matrix) # inverse of covariance matrix
        numerator = ones @ cov_inv @ mean_returns # # numerator of delta
        denominator = mean_returns.T @ cov_inv @ mean_returns #,  ones # denominator of delta
        delta = numerator / denominator
        w_tan.rename(columns = {'Tangency Portfolio' : 'Target Portfolio'}, inplace=True)
        return delta * target * w_tan # returns the vector of MV weights for each asset

Build the following portfolios:

#### Equally-weighted (EW)
Rescale the entire weighting vector to have target mean $\mutarg$. Thus, the $i$ element of the weight vector is,
$$\wEW_i = \frac{1}{n}$$

#### “Risk-parity” (RP)
Risk-parity is a term used in a variety of ways, but here we have in mind setting the weight of the portfolio to be proportional to the inverse of its full-sample variance estimate. Thus, the $i$ element of the weight vector is,
$$\wRP_i = \frac{1}{\sigma_i^2}$$

Note: we learned that in large portfolios the individual variances are irrelevant. Correlations and covariances are what matter

#### Mean-Variance (MV)
As described in `Section 2`.

In [ ]:
target_return = 0.01

#### Equally-Weighted Portfolio

# weights of equal weight portfolio (1/n)
weights_ew = np.ones(len(excess_returns_df.columns)) / len(excess_returns_df.columns)

# return of equally weighted portfolio
return_ew = excess_returns_df.mean() @ weights_ew

multiplier_ew = target_return / return_ew

# new equally weighted portfolio to achieve target return
scaled_ew = weights_ew * multiplier_ew

weights_df = pd.DataFrame(scaled_ew,
                   index = excess_returns_df.columns, 
                   columns = ['Equal-Weighted']
                   )


#### Risk-Parity Portfolio (adjust for risk in the form of variance)

# varaiances of each asset
vars = np.sqrt(np.diag(excess_returns_df.cov()))

# unscaled weights of each asset in risk parity portfolio
weights_rp = np.array(1 / vars)

# returns of risk parity portfolio
return_rp = excess_returns_df.mean() @ weights_rp

multiplier_rp = target_return / return_rp

# scaled risk-parity portfolio to fit target return
scaled_rp = weights_rp * multiplier_rp

weights_df['Risk-Parity'] = scaled_rp

#### MV Target Portfolio 

weights_target = mv_portfolio(excess_returns_df.mean(), 
                              excess_returns_df.cov(), 
                              target = 0.01)

weights_df['Target'] = weights_target


print('Portfolio Weights:')
display(weights_df)

Portfolio Weights:


,Equal-Weighted,Risk-Parity,Target
BWX,0.2546,0.379009,-0.635498
DBC,0.2546,0.188396,-0.011670
EEM,0.2546,0.178117,0.088674
EFA,0.2546,0.207934,0.040025
HYG,0.2546,0.413258,0.273724
IEF,0.2546,0.494593,0.936048
IYR,0.2546,0.186025,-0.247465
PSP,0.2546,0.147059,-0.178351
QAI,0.2546,0.639415,-1.414473
SPY,0.2546,0.219672,1.114643




### Comparing

In order to compare all these allocation methods, rescale each weight vector, such that it has targeted mean return of $\mutarg$.

* Calculate the performance of each of these portfolios over the sample.
* Report their mean, volatility, and Sharpe ratio. 
* How does performance compare across allocation methods?

In [ ]:
#Returns of the portfolios
ew_returns = (excess_returns_df @ scaled_ew).to_frame('Equally-Weighted')
rp_returns = (excess_returns_df @ scaled_rp).to_frame('Risk-Parity')
target_returns = (excess_returns_df @ weights_target)

# Summary stats
ew_returns_summary = performance_summary(ew_returns)
rp_returns_summary = performance_summary(rp_returns)
target_returns_summary = performance_summary(target_returns)

pd.concat([ew_returns_summary,
           rp_returns_summary,
           target_returns_summary],
           axis = 0).T

,Equally-Weighted,Risk-Parity,Target Portfolio
Annualized Mean,0.120000,0.120000,0.120000
Annualized Volatility,0.269666,0.262477,0.077334
Annualized Sharpe Ratio,0.444994,0.457183,1.551710


Note: equal weighted and risk-parity portfolios need to take on way more risk than the tangency portfolio (the target portfolio) to reach the same return. We should not be surprised about this result. The tangency was constructed to optimize the mean-variance trade-off. 

Note: also note that our risky weights don't add to one (i.e. we invest some of capital in risk free rate). We never have to keep track of risk free rate because we are dealing with excess returns

The annualized mean returns for the Equal-Weighted, Risk-Parity, and Target portfolios are all the same at 12% (0.01 monthly target). The Equal-Weight and Risk-Parity portfolios have very similar volatilities at ~27% while the target portfolio has volatility of 7.7334%. The sharpe ratios for the equal-weighted and risk-parity portfolios are also very similar at ~0.45 while the sharpe ratio of the MV Target Portfolio is 1.55171.

***

# 4. EXTRA: Out-of-Sample Performance

<i>This section is not graded, and you do not need to submit it. Still, we may discuss it in class, in which case, you would be expected to know it.<i>

Note: main idea is that you need to restimate, see how it performs, over and over. performance statistics for a period of time after the original portfolio weight calculations is lower than before. The summary statistics are changing

Note: changing the portfolio weights costs money (transaction costs) will bankrupt you. counting the transaction costs (even with a high return out of sample) will eat up the entire investment

Note: as you increase the number of assets to rebalance, the sharpe will go crazy bad

### 1. One-step Out-of-Sample (OOS) Performance
Let’s divide the sample to both compute a portfolio and then check its performance out of sample.
* Using only data through the end of `2022`, compute the weights built in Section 3.
* Rescale the weights, (using just the in-sample data,) to set each allocation to have the same mean return of $\mutarg$.
* Using those weights, calculate the portfolio’s Sharpe ratio within that sample.
* Again using those weights, (derived using data through `2022`,) calculate the portfolio’s OOS Sharpe ratio, which is based only on performance in `2023-2024`.

### 2. Rolling OOS Performance

Iterate the Out-of-Sample performance every year, not just the final year. Namely,
* Start at the end of `2015`, and calculate the weights through that time. Rescale them using the mean returns through that time.
* Apply the weights to the returns in the upcoming year, (`2016`.)
* Step forward a year in time, and recompute.
* Continue until again calculating the weights through `2023` and applying them to the returns in `2024`.

Report the mean, volatility, and Sharpe from this dynamic approach for the following portfolios:
* mean-variance (tangency)
* equally-weighted
* risk-parity
* regularized

***

# 5. EXTRA: Without a Riskless Asset

<i>This section is not graded, and you do not need to submit it. Still, we may discuss it in class, in which case, you would be expected to know it.

Re-do Section 2 above, but in the model without a risk-free rate.

That is, build the MV allocation using the two-part formula in the `Mean-Variance` section of the notes.
* This essentially substitutes the risk-free rate with the minimum-variance portfolio.
* Now, the allocation depends nonlinearly on the target mean return, $\mutarg$. (With a risk-free rate, we simply scale the weights up and down to achieve the mean return.)

You will find that, conceptually, the answers are very similar. 

***

# 6. EXTRA: Bayesian Allocation

Try the following allocation among the choices in `Section 3`...


#### Regularized (REG)
Much like the Mean-Variance portfolio, set the weights proportional to 
$$\wREG \sim \widehat{\Sigma}^{-1}\mux$$
but this time, use a regularized covariance matrix,
$$\widehat{\Sigma} = \frac{\Sigma + \Sigma_D}{2}$$
where $\Sigma_D$ denotes a *diagonal* matrix of the security variances, with zeros in the off-diagonals.

Thus, $\widehat{\Sigma}$ is obtained from the usual covariance matrix, $\Sigma$, but shrinking all the covariances to half their estimated values. 
